In [0]:
!wget http://www.gutenberg.org/files/11/11-0.txt

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers.recurrent import SimpleRNN
import numpy as np

In [0]:
fin=open('11-0.txt',encoding='utf-8-sig')
lines=[]
for line in fin:
  line = line.strip().lower()
  #line = line.decode("ascii","ignore")
  if(len(line)==0):
    continue
  lines.append(line)
fin.close()
text = " ".join(lines)

In [4]:
text[52:100]

', by lewis carroll this ebook is for the use of '

In [0]:
import re
text = text.lower()
text = re.sub('[^0-9a-zA-Z]+',' ',text)

In [0]:
from collections import Counter
counts = Counter()
counts.update(text.split())
words = sorted(counts, key=counts.get, reverse=True)
nb_words = len(text.split())
word2index = {word: i for i, word in enumerate(words)}
index2word = {i: word for i, word in enumerate(words)}

In [0]:
SEQLEN = 10
STEP = 1
input_words = []
label_words = []
text2=text.split()
for i in range(0,nb_words-SEQLEN,STEP):
    x=text2[i:(i+SEQLEN)]
    y=text2[i+SEQLEN]
    input_words.append(x)
    label_words.append(y)
print('input words list: ','\n',input_words[0])
print('label(output) words list: ','\n',label_words[0])

input words list:  
 ['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by']
label(output) words list:  
 her


In [0]:
total_words = len(set(words))
X = np.zeros((len(input_words), SEQLEN, total_words), dtype=np.bool)
y = np.zeros((len(input_words), total_words), dtype=np.bool)

In [0]:
for i, input_word in enumerate(input_words):
     for j, word in enumerate(input_word):
         X[i, j, word2index[word]] = 1
     y[i,word2index[label_words[i]]]=1
print('Shape of X: ',X.shape)
print('Shape of y: ',y.shape)

Shape of X:  (30407, 10, 3028)
Shape of y:  (30407, 3028)


In [0]:
HIDDEN_SIZE = 128
BATCH_SIZE = 32
NUM_ITERATIONS = 100
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [0]:
from keras.layers import LSTM

In [0]:
model = Sequential()
model.add(LSTM(HIDDEN_SIZE,return_sequences=False,input_shape=(SEQLEN,total_words)))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               1616384   
_________________________________________________________________
dense_2 (Dense)              (None, 3028)              390612    
Total params: 2,006,996
Trainable params: 2,006,996
Non-trainable params: 0
_________________________________________________________________


In [0]:
int(len(input_words)*0.1)

3040

In [0]:
len(input_words)

30407

In [0]:
for iteration in range(50):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION, validation_split = 0.1)
    test_idx = np.random.randint(int(len(input_words)*0.1)) * (-1)
    test_words = input_words[test_idx]
    print("Generating from seed: %s" % (test_words))
    for i in range(NUM_PREDS_PER_EPOCH):        
        Xtest = np.zeros((1, SEQLEN, total_words))
        for i, ch in enumerate(test_words):
            Xtest[0, i, word2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred,end=' ')
        test_words = test_words[1:] + [ypred]

Iteration #: 0
Train on 27366 samples, validate on 3041 samples
Epoch 1/1
27366/27366 [==============================] - 33s 1ms/step - loss: 6.2146 - val_loss: 8.2937
Generating from seed: ['not', 'met', 'the', 'solicitation', 'requirements', 'we', 'know', 'of', 'no', 'prohibition']
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the ==================================================
Iteration #: 1
Train on 27366 samples, validate on 3041 samples
Epoch 1/1
27366/27366 [==============================] - 34s 1ms/step - loss: 5.8628 - val_loss: 8.9148
Generating from seed: ['of', 'this', 'license', 'apply', 'to', 'copying', 'and', 'distributing',

In [0]:
from keras.layers import LSTM, Input, Bidirectional

In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(HIDDEN_SIZE,return_sequences=False),input_shape=(SEQLEN,total_words)))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 256)               3232768   
_________________________________________________________________
dense_3 (Dense)              (None, 3028)              778196    
Total params: 4,010,964
Trainable params: 4,010,964
Non-trainable params: 0
_________________________________________________________________


In [0]:
for iteration in range(50):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION, validation_split = 0.1)
    test_idx = np.random.randint(int(len(input_words)*0.1)) * (-1)
    test_words = input_words[test_idx]
    print("Generating from seed: %s" % (test_words))
    for i in range(NUM_PREDS_PER_EPOCH):        
        Xtest = np.zeros((1, SEQLEN, total_words))
        for i, ch in enumerate(test_words):
            Xtest[0, i, word2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred,end=' ')
        test_words = test_words[1:] + [ypred]

Iteration #: 0
Train on 27366 samples, validate on 3041 samples
Epoch 1/1
27366/27366 [==============================] - 60s 2ms/step - loss: 6.1460 - val_loss: 8.3666
Generating from seed: ['derive', 'from', 'the', 'use', 'of', 'project', 'gutenberg', 'tm', 'works', 'calculated']
the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle the turtle ==================================================
Iteration #: 1
Train on 27366 samples, validate on 3041 samples
Epoch 1/1
27366/27366 [==============================